In [ ]:
### Libraries ###
import numpy as np
import math as mt
import matplotlib.pyplot as plt
import scipy.linalg as lnalg

In [ ]:
### Parameters - Elements of mass and stiffness matrices ###
m = 0.06     # Mass
kd = 1    # Radial spring constant
kc = 1100000    # Circumferential coupling spring constant 

# Stiffness matrix elements
k0 = kd+2*kc 
k1 =-kc
k2 = 0
k3 = -kc

In [ ]:
### Creation of mass and stiffness matrix ###
n = 12   # Select number of blades

M = np.zeros((n, n))     # Create mass matrix(M) with dimension nxn containing zeros
np.fill_diagonal(M, m)   # Replace values on the diagonal of M with the variable m


a = np.zeros((1, n))     # Generate first two rows of stiffness matrix K
a[0,0] = k0
a[0,1] = k1
a[0,n-1] = k3
b = np.roll(a,1)
c = np.append(a, b, axis=0)


for i in range(n-2):     # Extend matrix c to n rows
    b = np.roll(b,1)
    c = np.append(c, b, axis=0)

K = c


In [ ]:
print("Mass matrix:")
print(M)
print()
print("Stiffness matrix:")
print(K)

In [ ]:
## Eigenfrequency analysis ##
# Solve the eigenproblem
eval, evec = lnalg.eigh(K, M)   #Calculate eigenvalues and eigenvectors
omegan =np.sqrt(eval)           #Calculate natural-frequency

# Print results
print("Eigenvalues and modal matrix (eigenvectors in columns):")
print(eval.round(decimals=3), evec.round(decimals=3), sep="\n\n")
print(" ")
print("Natural angular frequencies [rad/s]")
print(omegan.round(decimals=3))
print(" ")
print("Natural angular frequencies [Hz]")
f = omegan/2/np.pi
print(f.round(decimals=3))

In [ ]:
PHIn = evec/evec[:,0]
print("Modal matrix normalized with respect to Phi 1 (first eigenvector):")
print(PHIn.round(decimals=3))

In [ ]:
# Calculate modal system matrices - diagonalization, creates system of independent 1 dof systems
Mq = evec.T @ M @ evec
Kq = evec.T @ M @ evec

# Print matrices (no physical meaning)
print("Modal mass matrix:")
print(Mq.round(decimals=3))
print()
print("Modal stiffness matrix:")
print(Kq.round(decimals=3))

In [ ]:
## Define function for representing impeller displacement for all different modes ##
## Input number of blades, mode, normalized displacement vector, natural frequency ##
## The use of this function requires VPython to be installed ##

def ImpDisp(n_blades,mode, PHIn, omegan):
    silver = vector(0.75,0.75,0.75)
    ov = vector(0,0,0) 
    o = vertex(pos=ov, color=silver)
    ang_blade = pi/43
    l = 100
    az = vector(0,0,1)   
    a = pi/21 

    alpha = []
    beta = []
    gamma = []
    coorx1 = []
    coory1 = []
    coorx2 = []
    coory2 = []
    p1 = []
    p2 = []
    b = []
    al =[]
    be = []
    ga = []
    cox1 = []
    coy1 = []
    cox2 = []
    coy2 = []
    pt1 = []
    pt2 = []
    bd = []

    cone(pos=vector(0,0,0),
         axis=vector(0,0,20),
         radius=25, color=silver, texture=textures.metal)
    
    ## Draw fixed impeller (silver) ##
    for i in range(0,n_blades,1):

        alpha.append(i*2*pi/n_blades)
        beta.append(alpha[i] + ang_blade)
        gamma.append(alpha[i] - ang_blade)

        coorx1.append(l*cos(beta[i]))
        coory1.append(l*sin(beta[i])) 

        coorx2.append(l*cos(gamma[i]))
        coory2.append(l*sin(gamma[i]))        
        p1.append(vertex(pos=vec(coorx1[i],coory1[i],0),color=silver, texture=textures.metal))
        p2.append(vertex(pos=vec(coorx2[i],coory2[i],0),color=silver, texture=textures.metal))

        b.append(compound([triangle(vs=[o,p1[i],p2[i]])]))
 
    ## Draw displaced impeller (blue) ##
    for i in range(0,n_blades,1):

        al.append(i*2*pi/n_blades)
        be.append(al[i] + ang_blade)
        ga.append(al[i] - ang_blade)

        cox1.append(l*cos(be[i]))
        coy1.append(l*sin(be[i])) 

        cox2.append(l*cos(ga[i]))
        coy2.append(l*sin(ga[i]))        
        pt1.append(vertex(pos=vec(cox1[i],coy1[i],0),color=color.blue, opacity=1))
        pt2.append(vertex(pos=vec(cox2[i],coy2[i],0),color=color.blue, opacity=1))

        bd.append(compound([triangle(vs=[o,pt1[i],pt2[i]])]))
        
    for i in range(0,n_blades,1):
        bd[i].rotate(angle=a*PHIn[i,mode]*np.real(e**(1j*omegan[mode])), axis=az, origin=ov)

In [ ]:
from vpython import *
t = np.linspace(0,6,1000)

for ii in range(n):
    # Create in each iteration a scene with an impeller deformation for the corresponding mode shape
    scene = canvas(title='Relative displacements, mode '+str(ii), background=color.white, width=250, height=250)  
    scene.lights = []                                       
    local_light(pos=vector(0,0,50), color=color.white)
    ImpDisp(n,ii, PHIn, omegan)
    print("Natural frequency [Hz]= ")
    print(round(f[ii],3))
    plt.figure(ii)
    # Plot the position versus time of each blade, iterate for each mode
    for jj in range(n):
        plt.title("Position - Time, Mode "+str(ii))
        plt.xlabel("Time $[s]$")
        plt.ylabel("Relative Position")
        q = np.imag(PHIn[jj,ii]*np.e**(1j*omegan[ii]*t))
        plt.plot(t,q)
    plt.savefig("Mode"+str(ii)+".svg")
    #scene.capture("mode"+str(ii))


In [ ]:
mode = 0

scene1 = canvas(title='Vibration mode '+str(mode),background=color.white)
silver = vector(0.75,0.75,0.75)
scene.lights = []
##local_light(pos=vector(0,0,100), color=color.white)

ov = vector(0,0,0) 
o = vertex(pos=ov, color=silver)

ang_blade = pi/43
l = 100

az = vector(0,0,1)   
a = pi/215
n_blades = n
####################################################
alpha = []
beta = []
gamma = []
coorx1 = []
coory1 = []
coorx2 = []
coory2 = []
p1 = []
p2 = []
b = []

cone(pos=vector(0,0,0),
     axis=vector(0,0,20),
     radius=25, color=silver, texture=textures.metal)

for i in range(0,n_blades,1):
    
    alpha.append(i*2*pi/n_blades)
    beta.append(alpha[i] + ang_blade)
    gamma.append(alpha[i] - ang_blade)
    
    coorx1.append(l*cos(beta[i]))
    coory1.append(l*sin(beta[i])) 

    coorx2.append(l*cos(gamma[i]))
    coory2.append(l*sin(gamma[i]))        
    p1.append(vertex(pos=vec(coorx1[i],coory1[i],0),color=silver, texture=textures.metal))
    p2.append(vertex(pos=vec(coorx2[i],coory2[i],0),color=silver, texture=textures.metal))
    
    b.append(compound([triangle(vs=[o,p1[i],p2[i]])]))
#######################################################################
al =[]
be = []
ga = []
cox1 = []
coy1 = []
cox2 = []
coy2 = []
pt1 = []
pt2 = []
bd = []

for i in range(0,n_blades,1):
    
    al.append(i*2*pi/n_blades)
    be.append(al[i] + ang_blade)
    ga.append(al[i] - ang_blade)
    
    cox1.append(l*cos(be[i]))
    coy1.append(l*sin(be[i])) 

    cox2.append(l*cos(ga[i]))
    coy2.append(l*sin(ga[i]))        
    pt1.append(vertex(pos=vec(cox1[i],coy1[i],0),color=color.blue, opacity=1))
    pt2.append(vertex(pos=vec(cox2[i],coy2[i],0),color=color.blue, opacity=1))
    
    bd.append(compound([triangle(vs=[o,pt1[i],pt2[i]])]))



t = 0
dt=0.1

while True:
    rate(10)
    for i in range(0,n_blades,1):
        bd[i].rotate(angle=a*PHIn[i,mode]*np.real(e**(1j*omegan[mode]*t)), axis=az, origin=ov)
         
    t = t + dt